In [ ]:
# Import key librarys
import pandas as pd
import numpy as np

import os
import glob

import re

## Main code

In [ ]:
# # Extract organisation name and year from filenames

# rorgs = []
# ryears = []

# for f in xlsx_files:
    
#     # Get filename after / character
#     filename = f.split('/')[-1]
    
#     # Look for numbers in filename
#     num = re.findall(r'\d+', filename)
    
#     for n in num:
        
#         # If number more than 3 characters assume it is a year 
#         if len(n) > 3:
           
#             # Append to list of years
#             ryears.append(n)
    
#     # Look at filename before ' 2' and ' CCR' and assume is org
#     s = filename.split(' 2', 1)[0]
#     s = s.split(' - Climate Change Duties annual report', 1)[0]
#     s = s.split(' CCR', 1)[0]
    
#     # Append to list of orgs
#     rorgs.append(s)
    
# print(rorgs)
# print(ryears)

In [ ]:
def find_details(f):

    # Get filename after / character
    filename = f.split('/')[-1]
    
    # Look for numbers in filename
    num = re.findall(r'\d+', filename)
    
    for n in num:
        
        # If number more than 3 characters assume it is a year 
        if len(n) > 3:
           
            # Set year to n
            year = n
    
    # Look at filename before ' 2' and ' CCR' and assume is org
    s = filename.split(' 2', 1)[0]
    s = s.split(' - Climate Change Duties annual report', 1)[0]
    s = s.split(' CCR', 1)[0]
    
    # Set org to s
    org = s
    
    return org, year

# Creating a function which will remove extra leading and tailing whitespace from the data.
# Credit www.geeksforgeeks.org
def whitespace_remover(df):
   
    # iterating over the columns
    for i in df.columns:
         
        # checking datatype of each columns
        if df[i].dtype == 'object':
             
            # applying strip function on column
            df[i] = df[i].str.strip()
        
        else:
             
            # if conditition is False then it will do nothing.
            pass
        
def get_sheet(f):
    
    # Get list of sheet names
    sheet_names = pd.ExcelFile(f).sheet_names
    
    # Check type of template
    if len(sheet_names) > 5:
        
        # Define sheet number
        nsheet = 3
        
    else:
        
        nsheet = 0
        
    # Read in sheet 4
    df = pd.read_excel(f, sheet_name = sheet_names[nsheet])
    
    # Remove whitespace
    whitespace_remover(df)
        
    # Look for full match of header in df and return index
    header = 'Emission source'
    index = df[df.apply(lambda row: row.astype(str).str.fullmatch(header).any(), axis = 1)].index[0]
    
    # Read in sheet using header row
    df = pd.read_excel(f, sheet_name = sheet_names[nsheet], skiprows = index + 1, dtype = 'object')
    
    # Find location of useful columns
    a = df.columns.get_loc('Emission source')
    b = df.columns.get_loc('Scope')
    c = df.columns.get_loc('Consumption data')
    d = df.columns.get_loc('Units')
    e = df.columns.get_loc('Emission factor')
    f = df.columns.get_loc('Units.1')
    g = df.columns.get_loc('Emissions (tCO2e)')
    h = df.columns.get_loc('Comments')
    
    # Select important columns
    df = df.iloc[:, [a, b, c, d, e, f, g, h]]
    
    # Replace Blank values with NaN
    df = df.replace(r'^\s*$', np.nan, regex=True)
    
    return df
    
def sort_names_end(df):
    
    #
    
    # Remove any line breaks
    df = df.replace(r'\n',' ', regex = True) 
    #df = df.replace(r'\s+|\\n', ' ', regex=True) 
    
    # Remove whitespace
    whitespace_remover(df)
    
    # Replace dodgy substrings in Emission source column
    df['Emission source'] = df['Emission source'].str.replace('&amp;', '&')
    df['Emission source'] = df['Emission source'].str.replace('&;', '&')
    df['Emission source'] = df['Emission source'].str.replace('–', '-')
    df['Emission source'] = df['Emission source'].str.replace(' and ', ' & ')
    df['Emission source'] = df['Emission source'].str.replace('  ', ' ')
    
    df['Comments'] = df['Comments'].str.replace(';', ',', regex = False)
    
    df['Emission source'] = df['Emission source'].str.replace('average', 'Average')
    
    df['Emission source'] = df['Emission source'].str.replace(' (grey)', '', regex = False)
    
    df['Emission source'] = df['Emission source'].str.replace(' kWh', '')
    df['Emission source'] = df['Emission source'].str.replace('kWh', '')
    df['Emission source'] = df['Emission source'].str.replace(' tonnes', '')
    df['Emission source'] = df['Emission source'].str.replace(' litre', '')
    df['Emission source'] = df['Emission source'].str.replace(' litres', '')
    
    df['Emission source'] = df['Emission source'].str.replace(' km', '')
    df['Emission source'] = df['Emission source'].str.replace(' mileage', '')
    df['Emission source'] = df['Emission source'].str.replace(' miles', '')
    
    df['Emission source'] = df['Emission source'].str.replace('diesel', 'Diesel')
    df['Emission source'] = df['Emission source'].str.replace('hybrid', 'Hybrid')
    df['Emission source'] = df['Emission source'].str.replace('petrol', 'Petrol')
    
    # Replace dodgy substrings in Units column
    df['Units'] = df['Units'].replace('Klms', 'km')
    df['Units'] = df['Units'].replace('M2', 'm3')
    df['Units'] = df['Units'].replace('Miles', 'miles')
    df['Units'] = df['Units'].replace('Litres', 'litres')
    df['Units'] = df['Units'].replace('Tonnes', 'tonnes')
    
    # Replace dodgy substrings in Units column
    df['Units.1'] = df['Units.1'].replace('kgCO2e/tonne', 'kg CO2e/tonne')
    df['Units.1'] = df['Units.1'].replace('kgCO2e/km', 'kg CO2e/km')
    df['Units.1'] = df['Units.1'].replace('kg CO2e/ km', 'kg CO2e/km')
    df['Units.1'] = df['Units.1'].replace('kg CO2e/ mile', 'kg CO2e/mile')

    # Replace conditional formatting messages with NaN
    df = df.replace('Please select from drop down box', np.nan)
    #df = df.replace('Other (please specify in comments)', np.nan)

    # Replace dodgy names
    df = df.replace('Biomass Wood Chips', 'Biomass (Wood Chips)')
    df = df.replace('Biomass Wood Pellets', 'Biomass (Wood Pellets)')
    df = df.replace('Biomass (wood chips)', 'Biomass (Wood Chips)')
    df = df.replace('Biomass (Wood chips)', 'Biomass (Wood Chips)')
    df = df.replace('Diesel (100%) mineral Diesel', 'Diesel (100% mineral Diesel)')
    df = df.replace('LPGs', 'LPG')
    df = df.replace('Gas oil (buildings) Littlemill PS', 'Gas oil')
    df = df.replace('Gas oil (buildings) Sorn PS', 'Gas oil')
    df = df.replace('Burning Oil (Kerosene)s', 'Burning Oil (Kerosene)')
    
    df = df.replace('WEEE (mixed recycling)', 'WEEE (Mixed) Recycling')
    df = df.replace('Other WEEE -IT reuse', 'Other WEEE - IT reuse')
    
    df = df.replace('Refuse Municipal /Commercial /Industrial to Combustion', 
                        'Refuse Municipal & Commercial & Industrial to Combustion')
    df = df.replace('General waste to landfill', 
                        'Refuse Commercial & Industrial to Landfill')
    
    df = df.replace('International flights (Economy Class)', 'Long-haul flights (Economy Class)')
    df = df.replace('Domestic flight (Average passenger)', 'Short-haul flights (Economy Class)')
    
    # Correct orgs names
    df = df.replace('Abshire', 'Aberdeenshire Council')
    df = df.replace('Adeen City', 'Aberdeen City Council')
    df = df.replace('City of Edinburgh', 'City of Edinburgh Council')
    df = df.replace('CCPBD City of Edinburgh Council', 'City of Edinburgh Council')
    df = df.replace('Clackmananshire Council', 'Clackmannanshire Council')
    df = df.replace('Clacks', 'Clackmannanshire Council')
    df = df.replace('Comhairlie nan Eilean Siar', 'Comhairle nan Eilean Siar')
    df = df.replace('CNES W.Isles', 'Comhairle nan Eilean Siar')
    df = df.replace('Dumfries & Galloway council', 'Dumfries and Galloway Council')
    df = df.replace('Dundee', 'Dundee City Council')
    df = df.replace('Dundee City Council Public', 'Dundee City Council')
    df = df.replace('EDC', 'East Dunbartonshire Council')
    df = df.replace('East Dunbartonshire Council Report', 'East Dunbartonshire Council')
    df = df.replace('ELC', 'East Lothian Council')
    
    return df

In [ ]:
# use glob to get all the csv files in the folder
path = '/home/mtc/Documents/Code/highland_energy/ssn'
xlsx_files = glob.glob(os.path.join(path, "*.xlsx"))

# Create list for all dfs
frames = []

for f in xlsx_files:
    
    print(f)
    
    # Store organisation and year in variables
    org, year = find_details(f)
    
    # Get df from sheet
    df = get_sheet(f)
    
    # Look for NaN values in Emission source column
    null_index = df.loc[pd.isna(df["Emission source"]), :].index
    
    # Create a list
    ls = []
    
    # Loop through 5 values to make a list of step sizes in null_index
    for i in range(5):
        ls.append(null_index[i + 1] - null_index[i])
        
    # Define cut off
    cut_off = null_index[0 + ls.index(1)]

    # Drop All Rows After First Occurrence of two NaNs in a row
    df = df[:cut_off]
    
    # Add Columns with year and org
    # Subtract 1 from year as 2020 report is for 2019 etc.
    df['Year'] = int(year) - 1
    df['Organisation'] = org
    
    frames.append(df)

In [ ]:
# Concat frames together
dft = pd.concat(frames)

# Sort formatting
dft = sort_names_end(dft)

# Drop rows with more than 6 valid data cells
dft = dft.dropna(thresh = 6)

dft.to_csv('ssn_energy_total_summary_long.csv')

dft.head()

In [ ]:
#dft[(dft['Organisation'] == 'East Dunbartonshire Council') & (dft['Year'] == 2020)]

## Simplifying categorys and convert units to kWh

In [ ]:
# Make a copy of main df
dfk = dft.copy()

# Simplify names of Emission sources
dfk['Emission source'] = np.where(dfk['Emission source'].str.contains('Car', case = False
                                                                     ), 'Car', dfk['Emission source'])
dfk['Emission source'] = np.where(dfk['Emission source'].str.contains('HGV', case = False
                                                                     ), 'Lorry', dfk['Emission source'])
dfk['Emission source'] = np.where(dfk['Emission source'].str.contains('Van', case = False
                                                                     ), 'Van', dfk['Emission source'])
dfk['Emission source'] = np.where(dfk['Emission source'].str.contains('Ferry', case = False
                                                                     ), 'Ferry', dfk['Emission source'])
dfk['Emission source'] = np.where(dfk['Emission source'].str.contains('Flight', case = False
                                                                     ), 'Aeroplane', dfk['Emission source'])
dfk['Emission source'] = np.where(dfk['Emission source'].str.contains('Rail', case = False
                                                                     ), 'Rail', dfk['Emission source'])
dfk['Emission source'] = np.where(dfk['Emission source'].str.contains('Bus|Coach', case = False
                                                                     ), 'Bus', dfk['Emission source'])
dfk['Emission source'] = np.where(dfk['Emission source'].str.contains('Taxi', case = False
                                                                     ), 'Taxi', dfk['Emission source'])


dfk['Emission source'] = np.where(dfk['Emission source'].str.contains('Biomass', case = False
                                                                     ), 'Biomass', dfk['Emission source'])
dfk['Emission source'] = np.where(dfk['Emission source'].str.contains('Kerosene|Gas Oil|Fuel Oil', case = False
                                                                     ), 'Oil', dfk['Emission source'])
dfk['Emission source'] = np.where(dfk['Emission source'].str.contains('Renewable', case = False
                                                                     ), 'Renewables', dfk['Emission source'])


dfk['Emission source'] = np.where(dfk['Emission source'].str.contains('Landfill', case = False
                                                                     ), 'Landfill', dfk['Emission source'])
dfk['Emission source'] = np.where(dfk['Emission source'].str.contains('Recycling', case = False
                                                                     ), 'Recycling', dfk['Emission source'])
dfk['Emission source'] = np.where(dfk['Emission source'].str.contains('Composting|Organic', case = False
                                                                     ), 'Composting', dfk['Emission source'])


dfk['Emission source'] = np.where(dfk['Emission source'].str.contains('Diesel', case = False
                                                                     ), 'Diesel', dfk['Emission source'])
dfk['Emission source'] = np.where(dfk['Emission source'].str.contains('Petrol', case = False
                                                                     ), 'Petrol', dfk['Emission source'])


dfk['Emission source'] = np.where(dfk['Emission source'].str.contains('Water', case = False
                                                                     ), 'Water', dfk['Emission source'])

dfk.head()

In [ ]:
# Create lists to help with conservions
asources = ['Car', 'Bus', 'Bus', 'Rail', 'Lorry', 'Van', 'Taxi', 'Ferry', 
               'Oil', 'Diesel', 'Petrol', 'LPG', 'Biomass']

# Transport sources
tsources = ['Car', 'Bus', 'Bus', 'Rail', 'Lorry', 'Van', 'Taxi', 'Ferry']
# Other sources
osources = ['Oil', 'Diesel', 'Petrol', 'LPG', 'Biomass']

# Distance units
dunits = ['km', 'passenger km', 'miles']

# Transport Energy Use Figures from Sustainable Energy Without Hot Air p141
# Car - 68 kWh per 100 passenger-km
# Rail - 6 kWh per 100 passenger-km
# Bus - 19 kWh per 100 passenger-km
# Air - 51 kWh per 100 passenger-km
# Sea - 57 kWh per 100 passenger-km
#  Overall transport efficiencies of transport modes in Japan (1999).

# Function to convert into kWh
def calc_kWh(row):
    
    # If units already kWh its easy
    if row['Units'] == 'kWh':
        
        return row['Consumption data']
    
    if row['Units'] != 'kWh':
        
        # Check known source
        if row['Emission source'] in asources:
            
            # If a transport source
            if row['Emission source'] in tsources:
    
                # Create diction with kWh per 100 passenger-km
                kWhkm = {'Car':68, 'Rail':6, 'Bus':19, 'Air':51, 'Ferry': 57, 'Lorry':68, 'Van':68, 'Taxi':68}
                
                # If known units convert into 100 passenger-km, then kWh
                if row['Units'] in dunits:
                
                    if row['Units'] == 'km':

                        # Calculate number of 100 passenger-km (assume average car occupancy of 1.5)
                        pkm100 = row['Consumption data'] / 100 / 1.5

                    if row['Units'] == 'passenger km':

                        # Calculate number of 100 passenger-km
                        pkm100 = row['Consumption data'] / 100

                    if row['Units'] == 'miles':

                        # Calculate number of 100 passenger-km
                        pkm100 = row['Consumption data'] * 1.6 / 100 / 1.5
                        
                    # Output kWh
                    return kWhkm[row['Emission source']] * pkm100
                        
                else:
                    
                    # Unrecognised units
                    return np.nan
                
            # If a other source convert litres into kWh
            if row['Emission source'] in osources:
                
                # kWh per litre
                kWhl = {'Oil':10, 'Diesel':10, 'Petrol':10, 'LPG':10, 'Biomass':10}
                
                return kWhl[row['Emission source']] * row['Consumption data']

        else:
            # Unrecognised source
            return np.nan

# Run function on each row
dfk['Consumption data kWh'] = dfk.apply(lambda row: calc_kWh(row), axis = 1)

dfk.head()

In [ ]:
dfk.to_csv('ssn_energy_total_summary_long_kwh.csv')

In [ ]:
# #dfnk = dfk[dfk['Units'] != 'kWh']
# dfnk = dfk[dfk['Emission source'].isin(tsources)]

# #dfnk['Emission source'].value_counts()
# dfnk['Units'].value_counts()